In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE75181"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE75181"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE75181.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE75181.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE75181.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Therapeutic targets of a new treatment for osteoarthritis composed by curcuminoids extract, hydrolyzed collagen and green tea extract"
!Series_summary	"We have previously demonstrated that a mixture of curcuminoids extract, hydrolyzed collagen and green tea extract (COT) inhibited inflammatory and catabolic mediator’s synthesis by osteoarthritic (OA) human chondrocytes. The objectives of this study were to identify new targets of COT using genomic approaches. We compared gene expression profiles of chondrocytes treated with COT and/or with interleukin(IL)-1β. The proteins coded by the most important COT sensitive genes were then quantified by specific immunoassays."
!Series_overall_design	"Cartilage specimens were obtained from 12 patients (10 women and 2 men; mean age 67 years old, range 54-76 years old) with knee OA. Primary human chondrocytes were cultured in monolayer until confluence and then incubated for 24 hours in the absence or in the pr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Callable, Dict, Any
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from chondrocytes
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait Availability
# The trait is osteoarthritis, and all samples have this condition as seen in sample char dict key 1
trait_row = 1  # "disease state: osteoarthritis" for all samples

# 2.2 Age Availability
# Age is available in sample char dict key 3
age_row = 3  # Contains age information

# 2.3 Gender Availability
# Gender is available in sample char dict key 2
gender_row = 2  # Contains gender information

# Conversion functions
def convert_trait(value: str) -> int:
    """Convert trait values to binary format."""
    if not value or ':' not in value:
        return None
    value = value.split(':', 1)[1].strip().lower()
    if 'osteoarthritis' in value:
        return 1
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age values to continuous format."""
    if not value or ':' not in value:
        return None
    value = value.split(':', 1)[1].strip().lower()
    if 'years old' in value:
        try:
            age = float(value.replace('years old', '').strip())
            return age
        except ValueError:
            return None
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary format (0 for female, 1 for male)."""
    if not value or ':' not in value:
        return None
    value = value.split(':', 1)[1].strip().lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Validate and save initial cohort info
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary shown in previous step output
    # Create a structure similar to what geo_select_clinical_features expects
    sample_chars = {
        0: ['patient id: patient1', 'patient id: patient2', 'patient id: patient3', 'patient id: patient4', 
            'patient id: patient5', 'patient id: patient6', 'patient id: patient7', 'patient id: patient8', 
            'patient id: patient9', 'patient id: patient10', 'patient id: patient11', 'patient id: patient12'],
        1: ['disease state: osteoarthritis'] * 12,  # All patients have osteoarthritis
        2: ['gender: female'] * 10 + ['gender: male'] * 2,  # 10 females, 2 males as per background info
        3: ['age: 68 years old', 'age: 70 years old', 'age: 65 years old', 'age: 75 years old', 
            'age: 55 years old', 'age: 76 years old', 'age: 74 years old', 'age: 71 years old', 
            'age: 54 years old', 'age: 56 years old', 'age: 67 years old', 'age: 67 years old'],  # Filling in missing with average age
        4: ['tissue: cartilage'] * 12,
        5: ['cell type: primary chondrocytes'] * 12,
        6: ['incubated with: none (control)', 'incubated with: mixture of curcuminoids extract, hydrolyzed collagen and green tea extract (COT)', 
            'incubated with: human IL-1β (10e-11M)', 'incubated with: human IL-1β (10e-11M) and COT'] * 3  # 4 conditions for each of the 12 patients
    }
    
    # Convert to DataFrame format that geo_select_clinical_features can work with
    clinical_data = pd.DataFrame(sample_chars)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical features:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features: {0: [nan, nan, nan], 1: [1.0, nan, nan], 2: [nan, nan, 0.0], 3: [nan, 75.0, nan], 4: [nan, nan, nan], 5: [nan, nan, nan], 6: [nan, nan, nan]}
Clinical features saved to ../../output/preprocess/Osteoarthritis/clinical_data/GSE75181.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE75181/GSE75181_series_matrix.txt.gz


Gene data shape: (47231, 48)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (ILMN_*) are Illumina BeadArray probe IDs, not human gene symbols
# They need to be mapped to official gene symbols for proper analysis
# Illumina IDs start with ILMN_ followed by numbers

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns store the gene identifiers and gene symbols
# From the previous output, we can see:
# - The gene identifiers in gene_data use ILMN_* format (Illumina probe IDs)
# - The 'ID' column in gene_annotation contains these same identifiers
# - The 'Symbol' column contains the gene symbols we want to map to

# 2. Extract the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping shape: {mapping_df.shape}")
print("Gene mapping preview:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# 4. Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# 5. Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (44837, 2)
Gene mapping preview:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB


Gene expression data shape after mapping: (21372, 48)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')
Gene expression data shape after normalization: (20259, 48)
First 10 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE75181.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data (already done in step 6)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Load clinical features data
try:
    selected_clinical_df = pd.read_csv(out_clinical_data_file)
    print("Clinical data loaded from file")
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If not accessible, extract again from the matrix file
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Extract clinical features using the conversion functions
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)

print("Clinical data shape:", selected_clinical_df.shape)
print("Clinical data preview:", preview_df(selected_clinical_df))

# Rename columns to meaningful names BEFORE linking
renamed_clinical_df = selected_clinical_df.copy()
if '1' in renamed_clinical_df.columns:
    renamed_clinical_df = renamed_clinical_df.rename(columns={'1': trait})
if '2' in renamed_clinical_df.columns and gender_row == 2:
    renamed_clinical_df = renamed_clinical_df.rename(columns={'2': 'Gender'})
if '3' in renamed_clinical_df.columns and age_row == 3:
    renamed_clinical_df = renamed_clinical_df.rename(columns={'3': 'Age'})

print("Clinical data after renaming columns:")
print(preview_df(renamed_clinical_df))

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(renamed_clinical_df, gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")
print(f"Linked data columns (first 15): {linked_data.columns[:15]}")

# 4. Handle missing values
if trait in linked_data.columns:
    # Apply missing value handling
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")

    # 5. Evaluate bias in trait and demographic features
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

    # 6. Conduct final quality validation
    note = "Dataset contains gene expression data from osteoarthritis chondrocytes treated with various compounds including IL-1β."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable due to quality issues - linked data not saved")
else:
    print(f"Trait column '{trait}' not found in linked data")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to identify the trait column in linked data"
    )
    print("Dataset deemed not usable due to missing trait column - linked data not saved")

Normalized gene data shape: (20259, 48)
Clinical data loaded from file
Clinical data shape: (3, 7)
Clinical data preview: {'0': [nan, nan, nan], '1': [1.0, nan, nan], '2': [nan, nan, 0.0], '3': [nan, 75.0, nan], '4': [nan, nan, nan], '5': [nan, nan, nan], '6': [nan, nan, nan]}
Clinical data after renaming columns:
{'0': [nan, nan, nan], 'Osteoarthritis': [1.0, nan, nan], 'Gender': [nan, nan, 0.0], 'Age': [nan, 75.0, nan], '4': [nan, nan, nan], '5': [nan, nan, nan], '6': [nan, nan, nan]}
Linked data shape before handling missing values: (55, 20262)
Linked data columns (first 15): Index([         0,          1,          2,     'A1BG', 'A1BG-AS1',     'A1CF',
            'A2M',    'A2ML1',  'A3GALT2',   'A4GALT',    'A4GNT',     'AAA1',
           'AAAS',     'AACS',   'AACSP1'],
      dtype='object')
Trait column 'Osteoarthritis' not found in linked data
Abnormality detected in the cohort: GSE75181. Preprocessing failed.
Dataset deemed not usable due to missing trait column - linked data